# Configuration

Different variables can be defined here (eg. paths of data). These changes will be saved in the ```config.ini``` file and will therefore not have to be redefined each time. Note that other tabs can also modified this file.

In [1]:
import ipywidgets as widgets
import os

list_config_files = [x for x in os.listdir() if '.ini' in x]

config_w = widgets.Text(value=(list_config_files[0] if list_config_files != [] else ''),
                        description='Configuration file:', disabled=False)

widgets.VBox([config_w])


In [2]:
from datetime import datetime
from functions.config import Config

config_filename = config_w.value

if not os.path.isfile(config_filename):
    f = open(config_filename, "w")
    f.close()

config = Config(config_filename)
config.read()


path_database = widgets.Text(value=(config.get_value('database') if config.is_key('database') else ''),
                             description='Path of database:', disabled=False)

metadata_filename = widgets.Text(value=(config.get_value('metadata') if config.is_key('metadata') else ''),
                                 description='Metadata file:', disabled=False)

cutoff_date = widgets.DatePicker(description='Cut-off date:', value=(datetime.strptime(config.get_value('cutoff_date'), '%Y-%m-%d') if config.is_key('cutoff_date') else None), 
                                 disabled=False)

def on_button_save_clicked(var):
    config.set_value(path_database.value, 'database', section='PATH')
    config.set_value(metadata_filename.value, 'metadata', section='PATH')
    if cutoff_date.value is not None:
        config.set_value(str(cutoff_date.value), 'cutoff_date', section='OTHER')
    else:
        config.remove_value('cutoff_date', section='OTHER')
        
    config.write()

button_save = widgets.Button(description='Save "{}"'.format(config_filename), command=on_button_save_clicked)
button_save.on_click(on_button_save_clicked)

widgets.VBox([path_database, 
              widgets.HBox([metadata_filename, widgets.Label(value='This can be a relative path from the path of the database.')]),
              cutoff_date,
              button_save])